<a href="https://colab.research.google.com/github/Chandrasekhar1919/chandra_hds5210/blob/main/final_project/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Datasets:
texas alcohol consumption(Josn format) : https://data.texas.gov/dataset/Monthly-Per-Capita-Consumption/3j53-reqt/about_data

In this link i am using the api to get the json format access of the data.

Alcohol impaired driving deaths(csv format) : https://www.countyhealthrankings.org/health-data/texas?year=2024&measure=Alcohol-Impaired+Driving+Deaths&tab=1

I this link need to select the alcohol impaired deaths in the drop down bar.

In [2]:
import pandas as pd

LOADING DATA

In [6]:
data = pd.read_csv('/content/texas-2024-alcohol-impaired-driving-deaths-place-sort.csv')

df = pd.read_json('https://data.texas.gov/resource/3j53-reqt.json')


DISPLAYING THE DATA

In [7]:
data.head()

,County,# Alcohol-Impaired Driving Deaths,# Driving Deaths,County Value,Error Margin,National Z-Score
0,Texas,"4,874",25%,25-26%,NaN,"19,361"
1,Anderson,11,59,19%,12-26%,-0.58
2,Andrews,15,54,28%,21-35%,0.03
3,Angelina,20,78,26%,20-32%,-0.11
4,Aransas,2,25,8%,1-20%,-1.30


In [8]:
df.head()

,id,fiscalyear,year,distilledspiritstotaltax,month,population1,population2,distilledspiritstotalgallons,distilledspiritspercapit,distilledspiritstaxratep,...,highwinetaxratepergallon,sparklingwinetaxratepergallon,beertotaltax,beertotalgallons,beerpercapitaconsumption,beertaxratepergallon,aletotaltax,aletotalgallons,alepercapitaconsumption,aletaxratepergallon
0,200401,2004,2004,7588838.44,1,21120416,21372008,3162016,0.1497,2.4,...,0.408,0.516,9425512.50,48698579,2.3058,0.193548,440011.24,2222279,0.1052,0.198
1,200402,2004,2004,2858586.56,2,21120416,21372008,1191078,0.0564,2.4,...,0.408,0.516,7329422.21,37868757,1.7930,0.193548,370429.90,1870858,0.0886,0.198
2,200403,2004,2004,4035812.45,3,21120416,21372008,1681589,0.0796,2.4,...,0.408,0.516,6823183.80,35253187,1.6692,0.193548,359926.58,1817811,0.0861,0.198
3,200404,2004,2004,4623249.96,4,21120416,21372008,1926354,0.0912,2.4,...,0.408,0.516,8788742.30,45408593,2.1500,0.193548,430730.89,2175409,0.1030,0.198
4,200405,2004,2004,4357139.87,5,21120416,21372008,1815475,0.0860,2.4,...,0.408,0.516,9135415.74,47199742,2.2348,0.193548,433642.41,2190113,0.1037,0.198


CLEANING THE DATA

1.Clean # Driving Deaths:
Remove commas and percentages.
Convert to numeric values and replace NaN with 0.

2.Clean County Value:
For ranges (e.g., "25-26%"), calculate the average of the range.
Normalize the percentage to a decimal by dividing by 100.

3.Clean Error Margin:
Split ranges (e.g., "25-26%") into lower and upper bounds.
Convert to numeric and normalize by dividing by 100.

4.Clean National Z-Score:
Convert to numeric and replace any invalid values with 0.

5.Avoid inplace=True Warning:
Directly assign changes to columns rather than using inplace=True.

This ensures the dataset is clean, consistent, and ready for analysis.

In [11]:
# Clean # Driving Deaths: Remove commas and percentages, then convert to int
data['# Driving Deaths'] = data['# Driving Deaths'].replace({',': '', '%': ''}, regex=True)
data['# Driving Deaths'] = pd.to_numeric(data['# Driving Deaths'], errors='coerce')
data['# Driving Deaths'] = data['# Driving Deaths'].fillna(0)

# Clean County Value: Handle range values by converting them to the average percentage
def convert_range_to_mean(value):
    if '-' in value:  # Check if there's a range (e.g., '25-26')
        lower, upper = value.split('-')
        return (float(lower) + float(upper)) / 2
    else:
        return float(value)  # Handle cases like '25%'

data['County Value'] = data['County Value'].str.rstrip('%').apply(convert_range_to_mean) / 100

# Split Error Margin into lower and upper bounds
data[['Error Margin Lower', 'Error Margin Upper']] = data['Error Margin'].str.extract(r'(\d+)-(\d+)%')
data['Error Margin Lower'] = pd.to_numeric(data['Error Margin Lower'], errors='coerce') / 100
data['Error Margin Upper'] = pd.to_numeric(data['Error Margin Upper'], errors='coerce') / 100

# Fill NaN values in the National Z-Score column
data['National Z-Score'] = pd.to_numeric(data['National Z-Score'], errors='coerce').fillna(0)

# Print the cleaned data for inspection
print(data.head())

     County  # Alcohol-Impaired Driving Deaths  # Driving Deaths  \
0     Texas                               4874                25   
1  Anderson                                 11                59   
2   Andrews                                 15                54   
3  Angelina                                 20                78   
4   Aransas                                  2                25   

   County Value Error Margin  National Z-Score  Error Margin Lower  \
0         0.255          NaN              0.00                 NaN   
1         0.190       12-26%             -0.58                0.12   
2         0.280       21-35%              0.03                0.21   
3         0.260       20-32%             -0.11                0.20   
4         0.080        1-20%             -1.30                0.01   

   Error Margin Upper  
0                 NaN  
1                0.26  
2                0.35  
3                0.32  
4                0.20  


In [12]:
# Display basic information
print(df.info())

# Show summary statistics
print(df.describe())

# Check for missing values
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             245 non-null    int64  
 1   fiscalyear                     245 non-null    int64  
 2   year                           245 non-null    int64  
 3   distilledspiritstotaltax       245 non-null    float64
 4   month                          245 non-null    int64  
 5   population1                    245 non-null    int64  
 6   population2                    245 non-null    int64  
 7   distilledspiritstotalgallons   245 non-null    int64  
 8   distilledspiritspercapit       245 non-null    float64
 9   distilledspiritstaxratep       245 non-null    float64
 10  winetotaltaxcollections        245 non-null    float64
 11  winetotalgallonsestimated      245 non-null    int64  
 12  winepercapitaconsumption       245 non-null    flo

In [13]:
df.drop(columns=['fiscalyear', 'distilledspiritstaxratep', 'beertaxratepergallon', 'aletaxratepergallon'], inplace=True)

In [16]:
import numpy as np

df['population2'] = np.where(df['population2'] == 0, df['population1'], df['population2'])

In [17]:
from scipy.stats import zscore

df = df[(zscore(df['population2']) > -3) & (zscore(df['population2']) < 3)]

In [18]:
df['tax_per_gallon_spirits'] = df['distilledspiritstotaltax'] / df['distilledspiritstotalgallons']
df['tax_per_gallon_wine'] = df['winetotaltaxcollections'] / df['winetotalgallonsestimated']
df['tax_per_gallon_beer'] = df['beertotaltax'] / df['beertotalgallons']

In [19]:
print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             245 non-null    int64  
 1   year                           245 non-null    int64  
 2   distilledspiritstotaltax       245 non-null    float64
 3   month                          245 non-null    int64  
 4   population1                    245 non-null    int64  
 5   population2                    245 non-null    int64  
 6   distilledspiritstotalgallons   245 non-null    int64  
 7   distilledspiritspercapit       245 non-null    float64
 8   winetotaltaxcollections        245 non-null    float64
 9   winetotalgallonsestimated      245 non-null    int64  
 10  winepercapitaconsumption       245 non-null    float64
 11  avgwinerate                    245 non-null    float64
 12  lowwinetaxratepergallon        245 non-null    flo

In [20]:
# Example if `state` exists in `df` and `data`:
merged = df.merge(data, left_on='state_column', right_on='County', how='left')

KeyError: 'state_column'